# Classify Customer Support Tickets

Auto-categorize incoming requests by urgency and department.


In [ ]:
%pip install -qU pixeltable anthropic


In [ ]:
import os, getpass
if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass.getpass('Anthropic API Key:')


In [ ]:
import pixeltable as pxt
from pixeltable.functions import anthropic


In [ ]:
# Step 1: Create tickets table
pxt.create_dir('support', if_exists='ignore')
tickets = pxt.create_table('support.tickets', 
    {'ticket_id': pxt.String, 'subject': pxt.String, 'body': pxt.String}, if_exists='ignore')


In [ ]:
# Step 2: Auto-classify with LLM
tickets.add_computed_column(if_exists='ignore',
    classification=anthropic.messages(
        model='claude-3-haiku-20240307',
        max_tokens=100,
        messages=[{'role': 'user', 'content': 
            f"Classify this ticket:\nSubject: {tickets.subject}\nBody: {tickets.body}\n\n"
            f"Return JSON: {{\"category\": \"billing|technical|sales|other\", "
            f"\"urgency\": \"low|medium|high\", \"sentiment\": \"positive|neutral|negative\"}}"}]
    ).content[0].text
)


In [ ]:
# Step 3: Insert tickets (auto-classified)
tickets.insert([
    {'ticket_id': 'T001', 'subject': 'Cannot login', 'body': 'Getting error 500'},
    {'ticket_id': 'T002', 'subject': 'Billing question', 'body': 'Need refund'}
])


In [ ]:
# View results
tickets.select(tickets.ticket_id, tickets.subject, tickets.classification).collect()


**What's Happening:**
- Each ticket insert triggers LLM classification
- Structured JSON output for routing
- Can filter/sort by category or urgency
- Sentiment analysis included

**Variation:** Route to specific queues:
```python
high_urgency = tickets.where(
    tickets.classification.astype(pxt.Json)['urgency'] == 'high'
)
```

**Next:** `monitor-brand-sentiment-from-social-media.ipynb` • `answer-questions-from-docs.ipynb`
